In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

In [3]:
# Load the dataset
data = pd.read_csv('../.data/missing_values.csv')

In [4]:
# Convert features to numeric type
numerical_features = data.drop(['artists_id', 'track_id', 'artist_name', 'track_name', 'release_date', 'genres'], axis=1)
numerical_features = numerical_features.apply(pd.to_numeric, errors='coerce')

# Encode the genre labels
label_encoder = LabelEncoder()
encoded_genres = label_encoder.fit_transform(data['genres'])

# Split the dataset into one with missing genre data and one without
data_missing = data[data['genres'].isnull()]
data_complete = data[~data['genres'].isnull()]

# Split the complete dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(numerical_features, encoded_genres, test_size=0.2, random_state=42)

# Oversample the training data to balance the class distribution
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Train a HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Predict genres for songs with missing genres
empty_genre_indices = data_missing.index
empty_genre_features = numerical_features.loc[empty_genre_indices]
predictions = clf.predict(empty_genre_features)

# Decode predicted labels back to genre names
predicted_genres = label_encoder.inverse_transform(predictions)

# Fill in the missing genre columns with predicted genres
data.loc[empty_genre_indices, 'genres'] = predicted_genres

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

KeyboardInterrupt: 

: 